In [ ]:
from music21 import *
# from Continuo.kmeans_inversion import k_means_data
import numpy as np
from time import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import Continuo.ContinuoConverter as CC
from alfabeto_data.pickled_data import *
from alfabeto_sources import all_sources

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.metrics import silhouette_samples, silhouette_score

import pickle, threading, time, sys
from queue import Queue

In [ ]:
A = corpus.getComposer('palestrina')[0:400]
all_parsed = []
all_keys = []
chordified = []

In [ ]:
def crd_data(unparsed):  
    def parser(song):
        x = converter.parse(song)
        if type(x) is stream.Score:
            crd = x.chordify().flat.getElementsByClass(chord.Chord)
            key_sig = x.flat.getElementsByClass(key.KeySignature)[0].sharps
            bass_note = crd[-1].bass().pitchClass
            chordified.append(crd)
            all_keys.append((key_sig, bass_note))
        elif type(x) is stream.Opus:
            for y in x:
                crd = y.chordify().flat.getElementsByClass(chord.Chord)
                key_sig = y.flat.getElementsByClass(key.KeySignature)[0].sharps
                bass_note = crd[-1].bass().pitchClass
                chordified.append(crd)
                all_keys.append((key_sig, bass_note))
    start = time.time()
    for x in unparsed:
        st = time.time()
        parser(x)
        fin = time.time()
        print('finished', len(chordified), 'of', len(unparsed), 'in', fin - st, 'seconds')
    print('entire process took:', time.time() - start, 'seconds')

    palestrina_chords_pickle_out = open("pickles/temp/palestrina_chords_temp_1.pickle", "wb")
    pickle.dump((chordified, all_keys), palestrina_chords_pickle_out)
    palestrina_chords_pickle_out.close()

In [ ]:
crd_data(A)

In [ ]:
print(len(all_keys))

In [ ]:
def corpus_continuo(corpus_unparsed):
    st = time.time()
    all_continuo_data = []
    all_corpus_labels = []
    for x in corpus_unparsed:
        y = corpus.parse(x)
        both_parts = []
        harmony_notes = []
        bass_notes = []
        
        if type(y) is stream.Score:
            z = y.chordify().flat.getElementsByClass(chord.Chord)
            bass_note = z[-1].bass().pitchClass
            theKey = y.flat.getElementsByClass(key.KeySignature)
            all_corpus_labels.append((theKey[0].sharps, bass_note))
            for a in z:
                harmony = []
                root_note = (a.bass().pitchClass - bass_note)%12
                bass_notes.append(root_note)
                for b in a:
                    root_note = (a.bass().pitchClass - bass_note)%12
                    harmony.append((b.pitchClass - bass_note - root_note) %12)
                harmony_notes.append(sorted(set(harmony)))

        
        elif type(y) is stream.Opus:
            for yy in y:
                z = yy.chordify().flat.getElementsByClass(chord.Chord)
                bass_note = z[-1].bass().pitchClass
                theKey = yy.flat.getElementsByClass(key.KeySignature)
                all_corpus_labels.append((theKey[0].sharps, bass_note))
                
                for a in z:
                    harmony = []
                    bass_notes.append((a.bass().pitchClass - bass_note)%12)
                    for b in a:
                        harmony.append((b.pitchClass - bass_note - (a[-1].pitchClass - bass_note)%12))
                    harmony_notes.append(sorted(set(harmony)))
            for a, b in zip(bass_notes, harmony_notes):
                both_parts.append((a, b))

        for a, b in zip(bass_notes, harmony_notes):
            both_parts.append((a, b))

        all_continuo_data.append(both_parts)
    print(time.time() - st, 'seconds')
    return all_continuo_data, all_corpus_labels

In [ ]:
A = corpus_continuo(corpus.getComposer('palestrina')[0:5])

In [ ]:
bach_data = corpus_continuo(A)
bach_harm_pickle_out = open('pickles/bach_harm.pickle', 'wb')
pickle.dump(bach_data, bach_harm_pickle_out)
bach_harm_pickle_out.close()

bach_harm_pickle_in = open('pickles/bach_harm.pickle', 'rb')
bach_harm_data = pickle.load(bach_harm_pickle_in)

In [ ]:
def figure_sorter(list_tuple_data):
    final_data = []
    figures = [[0, 4, 7], [0, 3, 7], [0, 3, 8], [0, 4, 9], [0, 5, 9], [0, 5, 8]]
    for song in list_tuple_data:
        chord_number_total = len(song)
        temp_data = []
#         temp_data_other = []
        for figure in figures:
            figure_list = []
            for number in range(12):
                chord_number = 0
                for harmony in song:
                    if figure == harmony[1] and number == harmony[0]:
                        chord_number += 1

                if chord_number >0:
                    temp_data.append(chord_number/chord_number_total * 100)
                else:
                    temp_data.append(0)
        other_figures = []
        for number in range(12):
            number_other = 0
            for harmony in song:
                if harmony[0] == number and harmony[1] not in figures:
                    number_other += 1
            if number_other > 0:
                other_figures.append(number_other/chord_number_total*100)
            else:
                other_figures.append(0)
                   
        
        for x in other_figures:
            temp_data.append(x)
        final_data.append(temp_data)
    return final_data

In [ ]:
def inversion_data(corpus):
    continuo_corpus = []
    all_data = []
    for x in corpus:
        continuo_corpus.append(CC.book_converter_pc(x, 'all'))
    for w in continuo_corpus:

        for thing in w:
            frequency_dict = {}
            total_numbers = 0
            freq_list = []
            for number in range(12):
                for inversion_list in converters.triad_inversions_pc:
                    for inversion in inversion_list:
                        frequency_dict[str((number, inversion))] = 0
                        for x in thing:
                            if x[0] == number and x[1] == inversion:
                                frequency_dict[str((x[0], inversion))] += 1
                                total_numbers += 1
        # print(frequency_dict, total_numbers)
            for number in range(12):
                for inversion_list in converters.triad_inversions_pc:
                    for inversion in inversion_list:
                        for x, y in frequency_dict.items():
                            if x == str((number, inversion)):
                                # freq_list.append(y/total_numbers * 100)
                                freq_list.append(y/total_numbers * 100)
            # print(frequency_dict, total_numbers)
            all_data.append(freq_list)
    return all_data

In [ ]:
def bach_labeler(list_of_lists):
    bach_labels = []
    for x in list_of_lists[1]:
        bach_labels.append('$'+str(x[0])+':'+str(x[1])+'$')

In [ ]:
bach_data[0]

In [ ]:
def k_means_data(list_of_lists, cluster_number, label_markers):
    np.random.seed(42)

    digits = np.array(list_of_lists)
    # data = scale(digits)
    data = digits
    label_dict = {}
    label_numbers = []
    n = 0
    label_dict[label_markers[0]] = 0
    for j in range(1, len(label_markers)):
        if label_markers[j] not in label_dict:
            label_dict[label_markers[j]] = n+1
            n += 1

    for i in label_markers:
        label_numbers.append(label_dict[i])
    n_samples, n_features = data.shape
    n_digits = cluster_number
    labels = np.array(label_numbers)
    # print('label numbers', label_numbers)
    # sample_size = 300

    sample_size = len(data)
    # print(sample_size, 'Sample Size')


    print("n_digits: %d, \t n_samples %d, \t n_features %d"
          % (n_digits, n_samples, n_features))

    print(79 * '_')
    print('% 9s' % 'init'
                   '        time     inertia  homo     compl   v-meas  ARI  AMI   silhouette')

    def bench_k_means(estimator, name, data):
        t0 = time()
        estimator.fit(data)
        print('% 9s   %.2fs    %i   %.3f   %.3f   %.3f   %.3f   %.3f    %.3f'
              % (name, (time() - t0), estimator.inertia_,
                 metrics.homogeneity_score(labels, estimator.labels_),
                 metrics.completeness_score(labels, estimator.labels_),
                 metrics.v_measure_score(labels, estimator.labels_),
                 metrics.adjusted_rand_score(labels, estimator.labels_),
                 metrics.adjusted_mutual_info_score(labels, estimator.labels_),
                 metrics.silhouette_score(data, estimator.labels_,
                                          metric='euclidean',
                                          sample_size=sample_size)))


    bench_k_means(KMeans(init='k-means++', n_clusters=n_digits, n_init=10),
                  name="k-means++", data=data)

    bench_k_means(KMeans(init='random', n_clusters=n_digits, n_init=10),
                  name="random", data=data)

    # in this case the seeding of the centers is deterministic, hence we run the
    # kmeans algorithm only once with n_init=1
    pca = PCA(n_components=n_digits).fit(data)
    bench_k_means(KMeans(init=pca.components_, n_clusters=n_digits, n_init=1),
                  name="PCA-based",
                  data=data)
    print(79 * '_')

    ###############################################################################
    # Visualize the results on PCA-reduced data

    reduced_data = PCA(n_components=2).fit_transform(data)
    kmeans = KMeans(init='k-means++', n_clusters=n_digits, n_init=10)
    kmeans.fit(reduced_data)

    # Step size of the mesh. Decrease to increase the quality of the VQ.
    h = .02  # point in the mesh [x_min, m_max]x[y_min, y_max].

    # Plot the decision boundary. For that, we will assign a color to each
    x_min, x_max = reduced_data[:, 0].min() - 1, reduced_data[:, 0].max() + 1
    y_min, y_max = reduced_data[:, 1].min() - 1, reduced_data[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

    # Obtain labels for each point in mesh. Use last trained model.
    Z = kmeans.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.figure(1)
    plt.clf()
    plt.imshow(Z, interpolation='nearest',
               extent=(xx.min(), xx.max(), yy.min(), yy.max()),
               cmap=plt.cm.Paired,
               aspect='auto', origin='lower')

    for x, y, z in zip(reduced_data[:, 0], reduced_data[:, 1], label_markers):
        if z == '$Temperley MAJOR$':
            plt.plot(x, y, 'k.', marker='$M$', color='white', markeredgecolor='none', markersize=40)
        elif z == '$Temperley MINOR$':
            plt.plot(x, y, 'k.', marker='$m$', color='white', markeredgecolor='none', markersize=40)
        elif z[1] == '*':
            plt.plot(x, y, 'k.', marker=z, color='white', markeredgecolor='none', markersize=40)
        else:
            plt.plot(x, y, 'k.', marker=z, alpha=.3, markersize=len(z)*4)
#     plt.plot(reduced_data[:, 0], reduced_data[:, 1], 'k.')
        # plt.plot(x, y, 'k.')

    # for x in alf_labels:
    #         plt.plot(reduced_data[:, 0], reduced_data[:, 1], 'k.', marker = x, markersize=5)
    #     print('reduced data:', reduced_data)
    # Plot the centroids as a white X
    centroids = kmeans.cluster_centers_
    plt.scatter(centroids[:, 0], centroids[:, 1],
                marker='x', s=169, linewidths=3,
                color='w', zorder=10)
#     plt.title('K-means clustering of the corpus (PCA-reduced data)\n'
#               'Centroids are marked with white cross')
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    plt.xticks(())
    plt.yticks(())
    # plt.show()
    k_data = KMeans(init='k-means++', n_clusters=n_digits, n_init=10).fit(data)
    # return metrics.silhouette_score(data, k_data.labels_, metric='euclidean', sample_size=sample_size)
    return reduced_data, metrics.silhouette_score(data, k_data.labels_, metric='euclidean', sample_size=sample_size)

In [ ]:
# from alfabeto_data.pickled_data import *

def label_maker_alf(corpus_list):

#     final_labels = []
    alf_labels = []
#     for x in corpus_list:
#         for z in x.values():
#             print(z['data']['key'])
#             a = z['data']['key']
#             b = z['data']['final']
#             c = (a, b)
#             final_labels.append(c)
    
#     for x in final_labels:
    for x in corpus_list: #do this if the tuples are in list of list

        if x == ('f',2):
            alf_labels.append('$♭:D$')
        elif x == ('n',4):
            alf_labels.append('$♮:E$')
        elif x == ('n',9):
            alf_labels.append('$♮:A$')
        elif x == ('f',7):
            alf_labels.append('$♭:G$')
        elif x == ('n',2):
            alf_labels.append('$♮:D$')
        elif x == ('f',9):
            alf_labels.append('$♭:A$')
        elif x == ('f',0):
            alf_labels.append('$♭:C$')
        elif x == ('f',10):
            alf_labels.append('$♭:B♭$')
        elif x == ('n',5):
            alf_labels.append('$♮:F$')
        elif x == ('n',0):
            alf_labels.append('$♮:C$')
        elif x == ('f',5):
            alf_labels.append('$♭:F$')
        elif x == ('n',7):
            alf_labels.append('$♮:G$')
        else:
            alf_labels.append('o')
    return alf_labels

%matplotlib inline
for x in range(2, 7):
    k_means_data(figure_sorter(inversion_data(all_sources.GetAll.all_alf)), x, label_maker_alf(alfabeto_notes_data[1]))

In [ ]:
# bach_harm_pickle_out = open('pickles/bach_harm.pickle', 'wb')
# pickle.dump((figure_sorter(bach_harm_data[0]), bach_harm_data[1]), bach_harm_pickle_out)
# bach_harm_pickle_out.close()

# bach_harm_pickle_in = open('pickles/bach_harm.pickle', 'rb')
# bach_harm_data = pickle.load(bach_harm_pickle_in)

In [ ]:
bach_harm_data

In [ ]:
%matplotlib inline
# k_bach = k_means_data(figure_sorter(bach_data[0]), 2, bach_labels)

for x in range(2, 7):
    print(k_means_data(figure_sorter(bach_data[0]), x, bach_labels)[1])

In [ ]:
csv_ready = [sum(elem)/len(elem) for elem in zip(*figure_sorter(bach_data[0]))]

In [ ]:
len(figure_sorter(bach_data[0])[1])

In [ ]:
modal_labels = []
for x, y in zip(bach_data[0], bach_data[1]):
    if y == (-1, 5):
        modal_labels.append(x)

csv_ready

In [ ]:
csv_ready = [sum(elem)/len(elem) for elem in zip(*figure_sorter(modal_labels))]

In [ ]:
csv_readyR = []
for a in range(7):
    x = a*12
    csv_readyR.append(csv_ready[x:x+12])
    
    

In [ ]:
csv_readyR

In [ ]:
import csv

In [ ]:
with open('/home/daniel/Desktop/bach_continuo.csv', 'w', newline='') as fp:
    a = csv.writer(fp, delimiter=',')
    final_data = csv_readyR
    a.writerows(final_data)

In [ ]:
CSV

In [ ]:
print(u'\u266f')

In [ ]:
print(u'\u1D130'.encode)

In [ ]:
x = u'\u1D130'.encode('utf-8')

In [ ]:
x.decode('utf-8')

In [ ]:
import unicodedata

In [ ]:
unicodedata.lookup('FLAT')